This is a notebook to look at the catalog of sources from Johnny and find potential red field objects to get HI mass limits/detections for

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, Column
from astropy.io import ascii

In [9]:
file = '/Users/adams/python/udgs/arecibo-sample.csv'
cat=ascii.read(file,format='csv')
print(cat.colnames)
print(len(cat))

['cat-id', 'ra', 'dec', 'mu_0_i', 'm_tot_i', 'r_e', 'n', 'g-i', 'subsample']
20


In [12]:
#Use 100 Mpc as the fiducial distance
Mi = cat['m_tot_i'] - 5*np.log10(100.e6) + 5
Li = 10**((Mi - 4.3)/(-2.5))
#print(Li)

#convert to stellar mass
#Use Roediger & Corteau 2015
#i, g-i, FSPS
#log (M/L)λ = mλ ×(colour)+bλ
mi = 0.831 
bi = -0.597

ml_ratio = 10**( mi*cat['g-i'] +bi)

#now get the stellar masses:
mstar = ml_ratio * Li

#print(mstar)


In [30]:
#now get the HI masses I want to detect - set to be 0.1 Mstar
mhi = mstar

#and calculate integration time needed
#fiducial distance of 100 Mpc
fluxhi = mhi / (2.36e5 * 100.**2)

chanflux = fluxhi/50. #50 km/s width, flux level if evenly distributed over that 50 km/s

chanrms = chanflux/5.  #5sigma in each chan

gain = 8. #K/Jy
tsys = 30. #K
sefd = tsys/gain #Jy

chansize = 48.8e3 #Hz, 2048 chans over 25 MHz -> chanx4 for the ~10 km/s chan

tint = sefd**2 / (chansize * chanrms**2)

ttot = 2*tint

#print integration and total on source in minutes
print('Mstar    MHI         Flux  tint     t_tot  #time in minutes')
for i in range(len(tint)):
    print('{0:.2E}  {1:.2E} {4:6.3f} {2:6.1f}  {3:6.1f}'.format(mstar[i], mhi[i], tint[i]/60., ttot[i]/60.,fluxhi[i]))


Mstar    MHI         Flux  tint     t_tot  #time in minutes
4.81E+08  4.81E+08  0.204    7.2    14.4
1.43E+08  1.43E+08  0.061   81.3   162.7
1.95E+08  1.95E+08  0.082   44.1    88.3
7.95E+07  7.95E+07  0.034  264.4   528.9
2.24E+08  2.24E+08  0.095   33.4    66.8
3.00E+07  3.00E+07  0.013 1855.1  3710.1
4.92E+07  4.92E+07  0.021  692.0  1383.9
3.38E+08  3.38E+08  0.143   14.6    29.3
1.81E+08  1.81E+08  0.077   51.2   102.3
2.24E+07  2.24E+07  0.010 3317.4  6634.8
3.22E+07  3.22E+07  0.014 1612.5  3225.0
5.58E+07  5.58E+07  0.024  536.8  1073.5
9.19E+07  9.19E+07  0.039  198.1   396.2
5.84E+07  5.84E+07  0.025  490.7   981.3
1.01E+08  1.01E+08  0.043  163.7   327.4
6.02E+07  6.02E+07  0.026  460.8   921.5
1.49E+08  1.49E+08  0.063   75.6   151.1
6.48E+07  6.48E+07  0.027  398.1   796.3
1.08E+08  1.08E+08  0.046  143.3   286.6
2.49E+07  2.49E+07  0.011 2701.9  5403.7


In [40]:
#okay - things aren't going well. 
#What if I go the other way, and ask what I can do in an hour per source?
tint_set = 30*60. #on source time in seconds, assuming equal amount of time off-source

chanrms = sefd / np.sqrt(tint_set*chansize)

flux = chanrms*5.*50  #assume 5sigma, 50 km/s width

#then calculate HI mass and mhi/mstar
mhi_all = 2.36e5*100**2*flux  #assume 100 mpc

mhi_to_mstar = mhi_all / mstar

print('Mstar    MHI        MHI/Mstar  subsample')
for i in range(len(mstar)):
    print('{0:.2E}  {1:.2E} {2:5.2}  {3}'.format(mstar[i], mhi_all, mhi_all/mstar[i], cat['subsample'][i]))


Mstar    MHI        MHI/Mstar  subsample
4.81E+08  2.36E+08  0.49  galex
1.43E+08  2.36E+08   1.6  galex
1.95E+08  2.36E+08   1.2  galex
7.95E+07  2.36E+08   3.0  galex
2.24E+08  2.36E+08   1.1  galex
3.00E+07  2.36E+08   7.9  galex
4.92E+07  2.36E+08   4.8  isolated
3.38E+08  2.36E+08   0.7  isolated
1.81E+08  2.36E+08   1.3  isolated
2.24E+07  2.36E+08 1.1e+01  isolated
3.22E+07  2.36E+08   7.3  isolated
5.58E+07  2.36E+08   4.2  isolated
9.19E+07  2.36E+08   2.6  hst
5.84E+07  2.36E+08   4.0  hst
1.01E+08  2.36E+08   2.3  hst
6.02E+07  2.36E+08   3.9  hst
1.49E+08  2.36E+08   1.6  hst
6.48E+07  2.36E+08   3.6  hst
1.08E+08  2.36E+08   2.2  hst
2.49E+07  2.36E+08   9.5  hst


In [2]:
#Read the catalog
file = '/Users/adams/Downloads/tab-1-lsbg.csv'
cat=ascii.read(file,format='csv')

print(cat.colnames)
print(len(cat))
print(min(cat['r_e']),max(cat['r_e']))

redind = np.where(cat['g-i'] >= 0.64)
redcat = cat[redind]

['id', 'ra', 'dec', 'mu_0', 'mu_0_err', 'm_i', 'm_i_err', 'g-r', 'g-i', 'r_e', 'r_e_err', 'n', 'n_err', 'ell', 'ell_err', 'A_g', 'A_r', 'A_i']
781
2.58 13.86


In [7]:
print(min(redcat['m_i']))
print(max(redcat['m_i']))
print(np.mean(redcat['m_i']))

17.55
22.48
20.2194888889


In [4]:
#These sources have no distance, but I want to be clever about getting distance limits
#If I assume a maximum physical size, I can put a maximum distance to everything
#Say rmax = 6 kpc

distmax = 6e-3/(cat['r_e']/206265.)
dmaxred = 6e-3/(redcat['r_e']/206265.)

print(min(distmax),max(distmax))

ind = np.where(dmaxred <= 120.)[0]
print(len(ind))

print(redcat[ind])
print(redcat['g-r'][ind],redcat['g-i'][ind])

print(redcat['m_i'][ind])

89.2922077922 479.686046512
6
 id     ra      dec     mu_0 mu_0_err  m_i  ... ell  ell_err  A_g   A_r   A_i 
--- --------- -------- ----- -------- ----- ... ---- ------- ----- ----- -----
 18 243.88821 42.92376 24.13      0.4 19.09 ... 0.68    0.03  0.04 0.028  0.02
122  337.7865  1.16509 25.26      0.4 19.86 ...  0.2    0.04 0.234 0.166 0.118
363  29.59924 -5.63899 25.24      0.4 18.87 ... 0.31    0.03 0.071 0.051 0.036
425 224.90973  1.48513 25.67      0.4 19.52 ... 0.42    0.04 0.137 0.097 0.069
654 132.81382  0.25798 25.41      0.4 19.07 ... 0.49    0.03 0.099 0.071  0.05
680 222.31267  -0.7021  25.5      0.4  19.0 ... 0.63    0.03 0.132 0.094 0.067
g-r 
----
0.45
0.63
0.53
0.46
0.74
0.67 g-i 
----
0.66
 1.0
0.86
0.99
1.04
0.97
 m_i 
-----
19.09
19.86
18.87
19.52
19.07
 19.0


There are 6 sources that you would argue must be w/in 100 Mpc, assuming that these sources are instrinsically less than 6 kpc in size, and two are quite red

In [17]:
#what about a minimum size?
#Can I say galaxies are at least 1 kpc in size?
#Then what distance are they at?

distmin = 1e-3/(cat['r_e']/206265.)

print(distmin[ind])

#if my minimum distance is ~15 Mpc, what angular scale is 1.5 Mpc for searching?
print('I must search to {0} degrees at {1} Mpc'.format(1.5/15*206265/3600., 15.))
print('I must search to {0} degrees at {1} Mpc'.format(1.5/100*206265/3600., 100.))

     r_e     
-------------
15.7694954128
16.4616919393
 14.882034632
15.3356877323
16.1776470588
15.2112831858
I must search to 5.729583333333333 degrees at 15.0 Mpc
I must search to 0.8594375 degrees at 100.0 Mpc


The two red, large sources have a minimum distance of ~15 Mpc.

#425 doesn't appear isolated, but #680 may be